# Visualization of current COVID-19 trend in Shanghai

The visualization method is based on MinutePhysics' video [here](https://www.youtube.com/watch?v=54XLXg4fYsc).

Data depends on the work of [@kekincai](https://github.com/kekincai), downloaded from the deployed app available [here](https://kapaul.shinyapps.io/shanghai_covid19/).

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('全体数据一览.csv').drop_duplicates(subset=['date']).sort_values('date').reset_index()
dataset['新增本土确诊含无症状'] = dataset['新增本土新冠肺炎确诊病例'] + dataset['新增本土无症状感染者']
dataset

,index,Unnamed: 0,date,新增本土新冠肺炎确诊病例,新增本土无症状感染者,新增境外输入性新冠肺炎确诊病例,新增境外输入性无症状感染者,治愈出院,解除医学观察无症状感染者,新增本土确诊含无症状,无症状闭环隔离管控人数,无症状风险人群筛查人数,非管控区域病例比例
0,60,23.0,2022-03-01,1,1,37.0,17.0,0.0,0.0,2,0.0,1.0,100.000000
1,59,221.0,2022-03-03,2,14,43.0,21.0,0.0,0.0,16,0.0,14.0,100.000000
2,58,21.0,2022-03-04,3,16,24.0,10.0,0.0,0.0,19,0.0,16.0,100.000000
3,57,201.0,2022-03-05,0,28,25.0,10.0,0.0,0.0,28,0.0,28.0,100.000000
4,56,19.0,2022-03-06,3,45,32.0,16.0,0.0,0.0,48,0.0,45.0,100.000000
5,55,181.0,2022-03-07,4,51,36.0,10.0,27.0,10.0,55,0.0,51.0,100.000000
6,54,171.0,2022-03-08,3,62,26.0,10.0,32.0,7.0,65,0.0,62.0,100.000000
7,53,161.0,2022-03-09,4,76,42.0,16.0,18.0,13.0,80,0.0,76.0,100.000000
8,52,151.0,2022-03-10,11,64,32.0,10.0,17.0,7.0,75,0.0,64.0,100.000000
9,51,141.0,2022-03-11,5,78,22.0,9.0,20.0,3.0,83,0.0,78.0,100.000000


## Calculate our two axes

MinutePhysics recommeded using a moving average to counter the random fluctuations in daily new cases, so that's what we do here. We calculate a rolling average with a 3-day window.

The new cases are plotted not against **time**, but against **the total number of cases**. So we calculate the cumulated sum as well.

In [7]:
new_cases = dataset['新增本土确诊含无症状'].rolling(3).mean()
new_cases

0              NaN
1              NaN
2        12.333333
3        21.000000
4        31.666667
5        43.666667
6        56.000000
7        66.666667
8        73.333333
9        79.333333
10       74.333333
11       69.666667
12       88.333333
13      134.000000
14      200.333333
15      323.666667
16      509.000000
17      721.000000
18      878.333333
19      953.333333
20     1191.000000
21     1620.333333
22     2184.666667
23     2815.000000
24     3551.000000
25     4653.000000
26     5370.666667
27     5379.000000
28     5488.666667
29     6346.333333
30     7847.666667
31    10195.333333
32    13145.666667
33    16804.333333
34    19427.000000
35    21609.333333
36    23263.000000
37    24884.666667
38    24790.666667
39    25253.000000
40    25797.000000
41    25707.000000
42    24768.000000
43    23801.666667
44    23527.000000
45    22494.666667
46    20521.666667
47    19270.666667
48    18341.666667
49    19831.333333
50    20685.666667
51    21294.333333
52    19164.

In [8]:
all_cases = dataset['新增本土确诊含无症状'].cumsum()
all_cases.describe()

count        59.000000
mean     179343.457627
std      213941.344796
min           2.000000
25%        1452.500000
50%       50530.000000
75%      365745.000000
max      597775.000000
Name: 新增本土确诊含无症状, dtype: float64

In [9]:
vis = pd.DataFrame({
    'Day': dataset['date'],
    'New Cases': new_cases,
    'All Cases': all_cases
}).dropna()

vis

,Day,New Cases,All Cases
2,2022-03-04,12.333333,37
3,2022-03-05,21.000000,65
4,2022-03-06,31.666667,113
5,2022-03-07,43.666667,168
6,2022-03-08,56.000000,233
7,2022-03-09,66.666667,313
8,2022-03-10,73.333333,388
9,2022-03-11,79.333333,471
10,2022-03-12,74.333333,536
11,2022-03-13,69.666667,597


## Create the plot!

Here we use Plotly's built-in animation feature. Note that, in accordance to MinutePhysics' video, we're using logarithmic scales here for both X and Y axis.

I've added in a reference line (blue), as well as the overall trace (red).

In [10]:
import plotly.express as px
import plotly.graph_objects as go

X_AXIS = "All Cases"
Y_AXIS = "New Cases"

fig = px.scatter(vis, x=X_AXIS, y=Y_AXIS, 
                 animation_frame="Day",
                 range_x=[1, vis[X_AXIS].max() * 2], 
                 range_y=[1, vis[Y_AXIS].max() * 2], 
                 log_x=True, log_y=True,
                 color_discrete_sequence=['red'],
                 width=1000, height=700)

fig.update_layout(shapes = [{
    'type': 'line', 
    'yref': 'paper', 
    'xref': 'paper', 
    'y0': 0, 'y1': 1, 
    'x0': 0, 'x1': 1,
    'line': {
        'color': 'rgba(100, 100, 255, 0.5)'
    }
}])
fig.add_trace(go.Scatter(
    x=vis['All Cases'], 
    y=vis['New Cases'], 
    text=vis['Day'],
    mode='markers+lines+text',
    textposition="top left",
    textfont=dict(size=4),
    name="Shanghai"
))
fig.write_html("index.html")
fig.write_image("screenshot.png")
fig.show()